In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,10), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2))

In [3]:
def extract_keypoints(results):
    # Extract coordinates
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    mouth_left_x, mouth_left_y = pose[9*4], pose[9*4 + 1]
    shoulder_left_x, shoulder_left_y = pose[11*4], pose[11*4 + 1]
    shoulder_right_x, shoulder_right_y = pose[12*4], pose[12*4 + 1]
    elbow_left_x, elbow_left_y = pose[13*4], pose[13*4 + 1]
    elbow_right_x, elbow_right_y = pose[14*4], pose[14*4 + 1]
    nose_x, nose_y = pose[0], pose[1]

    left_thumb_x, left_thumb_y = lh[4*3], lh[4*3 + 1]
    right_thumb_x, right_thumb_y = rh[4*3], rh[4*3 + 1]
    right_index_x, right_index_y = rh[8*3], rh[8*3 + 1]
    left_index_x, left_index_y = lh[8*3], lh[8*3 + 1]
    right_middle_x, right_middle_y = rh[12*3], rh[12*3 + 1]
    left_middle_x, left_middle_y = lh[12*3], lh[12*3 + 1]
    right_ring_x, right_ring_y = rh[16*3], rh[16*3 + 1]
    left_ring_x, left_ring_y = lh[16*3], lh[16*3 + 1]
    right_pinky_x, right_pinky_y = rh[20*3], rh[20*3 + 1]
    left_pinky_x, left_pinky_y = lh[20*3], lh[20*3 + 1]

    right_hand_x, right_hand_y = rh[0], rh[1]
    left_hand_x, left_hand_y = lh[0], lh[1]

    distances = np.array([
        np.sqrt((mouth_left_x - left_thumb_x)**2 + (mouth_left_y - left_thumb_y)**2),
        np.sqrt((mouth_left_x - right_thumb_x)**2 + (mouth_left_y - right_thumb_y)**2),
        np.sqrt((left_thumb_x - right_thumb_x)**2 + (left_thumb_y - right_thumb_y)**2),
        np.sqrt((right_ring_x - left_ring_x)**2 + (right_ring_y - left_ring_y)**2),
        np.sqrt((right_middle_x - left_middle_x)**2 + (right_middle_y - left_middle_y)**2),
        np.sqrt((right_index_x - left_index_x)**2 + (right_index_y - left_index_y)**2),
        np.sqrt((right_pinky_x - left_pinky_x)**2 + (right_pinky_y - left_pinky_y)**2),
        np.sqrt((right_hand_x - left_hand_x)**2 + (right_hand_y - left_hand_y)**2),
        np.sqrt((right_hand_x - nose_x)**2 + (right_hand_y - nose_y)**2),
        np.sqrt((left_hand_x - nose_x)**2 + (left_hand_y - nose_y)**2),
        np.sqrt((nose_x - right_index_x)**2 + (nose_y - right_index_y)**2),
        np.sqrt((nose_x - left_index_x)**2 + (nose_y - left_index_y)**2),
        np.sqrt((nose_x - right_middle_x)**2 + (nose_y - right_middle_y)**2),
        np.sqrt((nose_x - left_middle_x)**2 + (nose_y - left_middle_y)**2),
        np.sqrt((shoulder_left_x - left_hand_x)**2 + (shoulder_left_y - left_hand_y)**2),
        np.sqrt((shoulder_right_x - right_hand_x)**2 + (shoulder_right_y - right_hand_y)**2),
        np.sqrt((elbow_right_x - right_hand_x)**2 + (elbow_right_y - right_hand_y)**2),
        np.sqrt((elbow_left_x - left_hand_x)**2 + (elbow_left_y - left_hand_y)**2),
        np.sqrt((elbow_left_x - nose_x)**2 + (elbow_left_y - nose_y)**2),
        np.sqrt((elbow_right_x - nose_x)**2 + (elbow_right_y - nose_y)**2)
    ])

    return np.concatenate([pose, lh, rh, distances])

In [4]:
DATA_PATH = r'C:\Users\Amrutha A\OneDrive\Desktop\Silcosys project\Distance_data'

actions = np.array(['a lot','abuse','afraid','you','free','bring','water','hiding','today', 'repeat', 'me', 'help', 'please'])

no_sequences = 30
sequence_length = 30
start_folder = 30

In [9]:
import os
import numpy as np

for action in actions: 
    action_path = os.path.join(DATA_PATH, action)
    
    if not os.path.exists(action_path):
        os.makedirs(action_path)
    
    # Get the maximum directory number within the action path
    dirmax = np.max(np.array(os.listdir(action_path)).astype(int)) if os.listdir(action_path) else 0
    
    for sequence in range(0, no_sequences):
        sequence_path = os.path.join(action_path, str(dirmax + sequence))
        
        if not os.path.exists(sequence_path):
            os.makedirs(sequence_path)


In [10]:
cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    for action in actions:
        for sequence in range(0, 30):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detection(frame, holistic)

                draw_styled_landmarks(image, results)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

                    cv2.imshow('OpenCV Feed', image)
                
                keypoints = extract_keypoints(results) #distance 
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [7]:
cap.release()
cv2.destroyAllWindows()

In [5]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

label_map = {label:num for num, label in enumerate(actions)}



In [6]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [7]:
print(np.array(sequences).shape)
X = np.array(sequences)
y = to_categorical(labels).astype(int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

(390, 30, 278)


In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, GRU, TimeDistributed 
from tensorflow.keras.callbacks import TensorBoard

In [9]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [12]:
model = Sequential()
model.add(GRU(64, return_sequences=True, activation='relu', input_shape=(30,278)))
model.add(GRU(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [13]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])

Epoch 1/200
12/12 [==============================] - 3s 16ms/step - loss: 2.4662 - categorical_accuracy: 0.1486
Epoch 2/200
12/12 [==============================] - 0s 15ms/step - loss: 2.2111 - categorical_accuracy: 0.2405
Epoch 3/200
12/12 [==============================] - 0s 16ms/step - loss: 1.9713 - categorical_accuracy: 0.2541
Epoch 4/200
12/12 [==============================] - 0s 17ms/step - loss: 1.6671 - categorical_accuracy: 0.3865
Epoch 5/200
12/12 [==============================] - 0s 17ms/step - loss: 1.3056 - categorical_accuracy: 0.5054
Epoch 6/200
12/12 [==============================] - 0s 15ms/step - loss: 1.0038 - categorical_accuracy: 0.6622
Epoch 7/200
12/12 [==============================] - 0s 16ms/step - loss: 0.6924 - categorical_accuracy: 0.7919
Epoch 8/200
12/12 [==============================] - 0s 15ms/step - loss: 0.4686 - categorical_accuracy: 0.8351
Epoch 9/200
12/12 [==============================] - 0s 14ms/step - loss: 0.4945 - categorical_accuracy:

In [14]:
print(model.summary())

res = model.predict(X_test)
print("Predicted: ",actions[np.argmax(res[1])])
print("Actual: ", actions[np.argmax(y_test[1])])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 30, 64)            66048     
                                                                 
 gru_3 (GRU)                 (None, 64)                24960     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 13)                429       
                                                                 
Total params: 97,677
Trainable params: 97,677
Non-trainable params: 0
_________________________________________________________________
None
1/1 [==============================] - 0s 258

In [15]:
model.save('distance_model_gru.h5')

In [17]:
from tensorflow.keras.models import load_model
model = load_model('distance_model_gru.h5')
model.load_weights('distance_model_gru.h5')

In [18]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

multilabel_confusion_matrix(ytrue, yhat)

1/1 [==============================] - 0s 215ms/step


array([[[18,  0],
        [ 0,  2]],

       [[18,  0],
        [ 1,  1]],

       [[18,  0],
        [ 0,  2]],

       [[16,  0],
        [ 0,  4]],

       [[18,  0],
        [ 0,  2]],

       [[19,  0],
        [ 0,  1]],

       [[19,  0],
        [ 0,  1]],

       [[19,  0],
        [ 0,  1]],

       [[16,  0],
        [ 0,  4]],

       [[18,  1],
        [ 0,  1]]], dtype=int64)

In [19]:
accuracy_score(ytrue, yhat)

0.95

In [20]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    #input_frame = cv2.resize(input_frame, (480, 640))
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        #cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [22]:
sequence = []
sentence = []
predictions = []
threshold = 0.55
simple = []
cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        
        ret, frame = cap.read()

        image, results = mediapipe_detection(frame, holistic)
        print(results)
        

        draw_styled_landmarks(image, results)

        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(actions[np.argmax(res)])
            predictions.append(np.argmax(res))
    
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])
                    simple = sentence
            if len(simple) > 5: 
                simple = simple[-5:]
            image = prob_viz(res, actions, image, colors)            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(simple), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        cv2.imshow('OpenCV Feed', image)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()
#print(predictions)
    #fps
    #fps drop
    #lstm

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti